Create a model using built-in library from Pytorch.
This code closely follows nn_tutorial notebook.

Multiple hyperhapameter combinations are tried to search for a best model. Mount the book to the Google drive if want to use Google GPU cloud.

In [ ]:
import os
from google.colab import drive
drive.mount('/content/gdrive')
!pwd
os.chdir('gdrive/My Drive/PyHack2019/')
!pwd
!ls

In [ ]:
import csv, torch, math, os, pickle
from torch import nn
from torch import optim
import torch.nn.functional as F

global chroma_shape, epochs, train_bs, validate_bs, lr, n_class

if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'
  
print(device)

In [ ]:
## read attr and tar in .pkl files
## Param: ratio = training size to the sample size. 
##         E.g. 0.7 means using 70% of the samples as training 
##              sample and the rest as validation sample. 
## Return: (1) a list of padded attr arrays
##         (2) a list of paddrd tar arrays
def read_data(ratio):
    
    assert (ratio > 0 and ratio < 1), 'Invalid ratio'
    
    
    att_file = open(r'data/eight_mfcc_noise_attr.pkl', 'rb')
    x_train = pickle.load(att_file)  
    att_file.close()
    
    tar_file = open(r'data/eight_mfcc_noise_tar.pkl', 'rb')
    str_y_train = pickle.load(tar_file)
    tar_file.close()
    
    y = []
    class_map = ['c', 'ch', 'q', 's', 'sh', 'x', 'z', 'zh']
    for s in str_y_train:
        if s in class_map:
            y.append(class_map.index(s))


    y = torch.tensor(y)
    
    assert (len(x_train) == len(y)), 'Unequal sample lengths'
    
    x = torch.tensor(x_train).float() 
    divider = int(len(y)*ratio)
       
    x_train = x[:divider, :, :]
    y_train = y[:divider]


    x_validate = x[divider:, :, :]
    y_validate = y[divider:]

    assert (x_train.shape[0] + x_validate.shape[0] == len(y_train) + len(y_validate)), 'Lengths do not add up'

    return x_train, y_train, x_validate, y_validate

In [ ]:
## create the train and the validation sets from processed file.
x_train, y_train, x_validate, y_validate = read_data(0.7)

x_train = x_train.to(device)
y_train = y_train.to(device)
x_validate = x_validate.to(device)
y_validate = y_validate.to(device)

print (x_train.device)

In [ ]:
## the number of classes
n_class = 8

## Get the shape of a padded instance for model construction
chroma_shape = x_train[0].shape

n_train = x_train.shape[0]
n_validate = x_validate.shape[0]
loss_func = F.cross_entropy

In [ ]:
def accuracy(out, yb):
    ##get the index with the max
    preds = torch.argmax(out, dim = 1)
    return (preds == yb).float().mean()

Since there are three classes, we set D_out to 3. n is total number of instances and c is the number of attributes in each instance. We use a loss function from torch.nn.functional.

In [ ]:
## Create a network class
## Create one class and share with model_CNN_classify?
class SoundRecognition_CNN(nn.Module):
  
    def __init__(self, dropout_rate):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=0)
        self.conv2 = nn.Conv2d(16, 16, kernel_size=3, stride=1, padding=0)
        self.conv3 = nn.Conv2d(16, 16, kernel_size=3, stride=1, padding=0)
        
        self.dropout = nn.Dropout(p = dropout_rate)
        
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = torch.nn.Linear(16*3*116, n_class)

    def forward(self, xb):
        
        xb = xb.view(-1, 1, chroma_shape[0], chroma_shape[1])

        xb = self.dropout(F.relu(self.conv1(xb)))
        xb = self.dropout(F.relu(self.conv2(xb)))
        xb = self.dropout(F.relu(self.conv3(xb)))

        xb = self.pool(xb)

        xb = xb.view(-1, 16*3*116)

        xb = self.fc1(xb)
        return xb.view(-1, xb.size(1))

## Get the model and optim object that will be used to update model parameters
def get_model(dropout_rate, weight_decay, lr):
    model = SoundRecognition_CNN(dropout_rate)
    return model, optim.Adam(model.parameters(), weight_decay = weight_decay, lr = lr)

In [ ]:
## PARAM: trained_model = trained model
##        validate_bs = a batch of validation data
##        n_validate = 
def validate(trained_model, validate_bs, n_validate):
    trained_model.eval()
    loss = []
    acc = []
    with torch.no_grad():

        for i in range((n_validate - 1) // validate_bs + 1):
            start_i = i * validate_bs
            end_i = start_i + validate_bs
            #print(xb.shape)
            xb = x_validate[start_i:end_i, :, :]
            yb = y_validate[start_i:end_i]
            pred = trained_model(xb)
            loss.append(loss_func(pred, yb))
            acc.append(accuracy(pred, yb))

        #print(loss)
        valid_loss = sum(loss)
    return valid_loss, sum(acc)/len(acc)

In [ ]:
def fit(epochs = 100, 
        train_bs = 10, 
        n_train = n_train, 
        validate_bs = 20, 
        n_validate = n_validate,
        dropout_rate = 0.5, 
        weight_decay = 1e-6, 
        lr = 1e-4,
        tolerance = 0.05):
  
  
    model, opt = get_model(dropout_rate = dropout_rate, weight_decay = weight_decay, lr = lr)
    model.to(device)

    ## to save training progress
    text_file = ''
    prev_small_vl = 10000000
    for epoch in range(epochs):

        #print('Training')
        model.train()
        for i in range((n_train - 1) // train_bs + 1):

            start_i = i * train_bs
            end_i = start_i + train_bs
            xb = x_train[start_i:end_i, :, :]
            yb = y_train[start_i:end_i]
            pred = model(xb)
            #print('pred: ', pred, ' | yb: ', yb)
            loss = loss_func(pred, yb)

            loss.backward()
            opt.step()
            opt.zero_grad()

        valid_loss, acc = validate(model, validate_bs, n_validate)

        ##early stop
        if valid_loss < prev_small_vl:
            prev_small_vl = valid_loss
        elif valid_loss > prev_small_vl*(1+tolerance):
            print('Epoch: ', epoch, ' == Early stop due to an increase in validation loss')
            break

    ## periodically report progress
    if (epoch + 1)% 10 == 0:
        print('Epoch: ', epoch + 1, ' | Loss: ', valid_loss, ' | Accuracy: ', acc)
        text_file = text_file + 'Epoch: '+ str(epoch + 1) + ' | Loss: ' + str(valid_loss) + ' | Accuracy: ' + str(acc) + '\n'
    text_file = str(model) + '\n\n' + text_file
    print('Train Finished')
    return model, text_file

In [ ]:
##generate informative name for hyperparameter combinations
def getName(epochs, dropout_rate, weight_decay, lr):
    name = 'mfcc_m_' + str(epochs) + '_dr' + str(dropout_rate) + '_wc' + str(weight_decay) + '_lr' + str(try_lr)
    return name +'.pt', name +'.txt'

In [ ]:
###### Make change under this this line########

###############################################


### run model with several sets of hyperparameters
### the models and training results are saved

## specify the directory to save the models
model_dir = 'model8/'
result_dir = 'result8/'

wc_list = [1e-6]
dr_list = [0.5, 0.8]
lr_list = [1e-4, 1e-5, 1e-6]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

# wc_list = [1e-5]  #[1e-3, 1e-4, 1e-5]
# dr_list = [0, 0.1, 0.5, 0.75, 0.9] #[0, 0.1, 0.5, 0.75, 0.9]
# lr_list = [1e-4, 1e-5, 1e-6]   #no wc -4 w/ dr .75 and dr .9 at all lr
epochs = 10
#dropout_rate = 0.5
#weight_decay = 1e-6

for try_wc in wc_list:
    for try_dr in dr_list:
        for try_lr in lr_list:
            m_save_path, f_save_path = getName(epochs, try_dr, try_wc, try_lr)
            print('Training: ', m_save_path)
            m, t_file = fit(epochs = epochs, 
                      train_bs = 100, 
                      n_train = n_train, 
                      validate_bs = 200, 
                      n_validate = n_validate,
                      dropout_rate = try_dr, 
                      weight_decay = try_wc, 
                      lr = try_lr,
                      tolerance = 0.1)
            print('DONE')

#             torch.save(m, model_dir + m_save_path)
#             file = open(result_dir + f_save_path,"w") 
#             file.write(t_file)
#             file.close()